In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import matplotlib.cm as cm
from scipy.misc import imresize
import cv2
import pandas as pd
import scipy.io as io

C:\Anaconda-python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def data_extraction(path):
    data_l, data_r, data_all = [], [], []
    nor_data = io.loadmat(path)
    temp = nor_data['data']
    r = temp['right']
    l = temp['left']
    r_temp = r[0,0]
    l_temp = l[0,0]
    data_r.append(r_temp['image'][0][0])
    data_r.append(r_temp['gaze'][0][0])
    data_r.append(r_temp['pose'][0][0])
    data_l.append(l_temp['image'][0][0])
    data_l.append(l_temp['gaze'][0][0])
    data_l.append(l_temp['pose'][0][0])
    for i in range(3):
        data_all.append(data_r[i])   #0 -> r_img, 1-> l_img, 2-> r_gaze, 3->l_gaze..
        data_all.append(data_l[i])
    return data_all

In [3]:
def label_extraction(path):
    txt = pd.read_csv(path, sep = ' ', header= None)
    df_anno = pd.DataFrame(txt)
    data_frame = pd.DataFrame()
    data_frame['Screen_x'] = df_anno[:][24]
    data_frame['Screen_y'] = df_anno[:][25]
    return data_frame

In [4]:
data_test = data_extraction('C:/MPIIGaze/MPIIGaze/Data/Normalized/p00/day01.mat')
label_text = label_extraction('C:/MPIIGaze/MPIIGaze/Data/Original/p00/day01/annotation.txt')

In [5]:
def make_dataset(image_shape, label_shape, image_list, label_list):
    initial_image_array_left = np.zeros([len(image_list[0]),image_shape])
    initial_image_array_right = np.zeros([len(image_list[0]),image_shape])
    initial_label_array = np.zeros([len(image_list[0]),label_shape])
    for n in range(len(image_list[0])):
        initial_image_array_left[n, :] = image_list[1][n].reshape(image_shape)
        initial_image_array_right[n, :] = image_list[0][n].reshape(image_shape)
        initial_label_array[n, :] = [label_list['Screen_x'][n], label_list['Screen_y'][n]]
    return initial_image_array_right, initial_image_array_left, initial_label_array

In [6]:
dataset_data_r, dataset_data_l, dataset_label = make_dataset(36 * 60, 2, data_test, label_text)

In [7]:
def model_for_image(X_img, keep_prob, name):
    W1 = tf.Variable(tf.random_normal([4, 4, 1, 5], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.relu(L1)

    print(L1)

    W2 = tf.Variable(tf.random_normal([2, 2, 5, 10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    L2_flat = tf.reshape(L2, [-1, 5 * 8 * 10])
    
    print("'L2's shape",L2.shape)
    print(L2_flat)


    W3 = tf.get_variable("W3_"+name ,shape= [400, 256],
                         initializer= tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([256]))
    L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

    print(L3.shape)

    W4 = tf.get_variable("W4_"+name ,shape= [256, 64],
                         initializer= tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([64]))
    logits = tf.matmul(L3, W4) + b4
    
    return logits

In [8]:
learning_rate = 0.001

Y = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)

X_left = tf.placeholder(tf.float32, [None, 2160])
X_right = tf.placeholder(tf.float32, [None, 2160])
X_img_left = tf.reshape(X_left, [-1, 36, 60, 1])   
X_img_right = tf.reshape(X_right, [-1, 36, 60, 1]) 

left_eye_logits = model_for_image(X_img_left, keep_prob, 'left')
right_eye_logits = model_for_image(X_img_right, keep_prob, 'right')

Tensor("Relu:0", shape=(?, 18, 30, 5), dtype=float32)
'L2's shape (?, 5, 8, 10)
Tensor("Reshape_2:0", shape=(?, 400), dtype=float32)
(?, 256)
Tensor("Relu_3:0", shape=(?, 18, 30, 5), dtype=float32)
'L2's shape (?, 5, 8, 10)
Tensor("Reshape_3:0", shape=(?, 400), dtype=float32)
(?, 256)


In [9]:
print(right_eye_logits.graph)
print(left_eye_logits.graph)

In [10]:
W1_le = tf.get_variable("W1_le" ,shape= [64, 16],
                         initializer= tf.contrib.layers.xavier_initializer())

print(W1_le.graph)

W1_re = tf.get_variable("W1_re" ,shape= [64, 16],
                         initializer= tf.contrib.layers.xavier_initializer())

print(W1_re.graph)

b1_le = tf.Variable(tf.random_normal([16]))

print(b1_le.graph)

b1_re = tf.Variable(tf.random_normal([16]))

print(b1_re.graph)

FC_model_1 = tf.matmul(left_eye_logits, W1_le) + tf.matmul(right_eye_logits, W1_re)
print(FC_model_1.graph)

W_fc = tf.get_variable("W_fc" ,shape= [16, 2],
                         initializer= tf.contrib.layers.xavier_initializer())
b_fc = tf.Variable(tf.random_normal([2]))
logits = tf.matmul(FC_model_1, W_fc) + b_fc

cost = tf.reduce_mean(tf.square(logits-Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')
for i in range(len(dataset_data_l)):
    feed_dict = {X_right: dataset_data_r,X_left: dataset_data_l,
                 Y:dataset_label, keep_prob: 0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
         print('step: {0}, cost: {1}'.format(i, c))

print('Learning Finished!')

Learning started. It takes sometime.
step: 0, cost: 381022.90625


KeyboardInterrupt: 

In [12]:
print('W_' + a)

NameError: name 'a' is not defined

In [ ]:
W1 + a = 4